In [ ]:
pip install entropy-pooling

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving RETURNS_new.csv to RETURNS_new.csv


In [ ]:
import pandas as pd
import numpy as np
from entropy_pooling import ep


col_indices = [20, 21, 22, 23, 24]
df = pd.read_csv('RETURNS_new.csv', usecols=col_indices)
df=df.dropna().iloc[1:]# Drop the first row with headers and any NaN values

df.columns = ['GOOGL', 'AAPL', 'ADBE', 'META', 'MSFT']

for col in df.columns:
    df[col] = df[col].str.rstrip('%').astype('float') / 100
    returns_df =df
# Convert DataFrame to NumPy array
R = returns_df.values.astype(float)
S = R.shape[0]
p = np.ones((S, 1)) / S

# Calculate prior mean and volatility
means_prior = p.T @ R
vols_prior = np.sqrt(p.T @ (R - means_prior)**2)

# Defining the views
views = {
    'AAPL_mean': 0.21 / 100,  # Converting percentage to decimal
    'MSFT_vol': 1.5 / 100,    # Converting percentage to decimal
    'GOOGL_mean': 0.53 / 100, # Converting percentage to decimal
    'META_vol': 2.51 / 100    # Converting percentage to decimal
}


A = np.vstack((
    np.ones((1, S)),
    R[:, 1],
    R[:, 0]
))

b = np.array([1, views['AAPL_mean'], views['GOOGL_mean']]).reshape(-1, 1)

G = np.vstack((
    (R[:, 4] - means_prior[0, 4])**2,
    (R[:, 3] - means_prior[0, 3])**2
))

h = np.array([views['MSFT_vol']**2, views['META_vol']**2]).reshape(-1, 1)

# Calculating posterior probabilities using entropy pooling
posterior_prob = ep(p, A, b, G, h)

# Calculating posterior mean
posterior_mean = (posterior_prob.T)*100 @ R

# Calculating posterior volatility for each asset
posterior_vols = np.zeros(R.shape[1])
for i in range(R.shape[1]):
    squared_deviations = (R[:, i] - (posterior_mean[0, i] / 100))**2
    posterior_vols[i] = np.round(np.sqrt(posterior_prob.T @ squared_deviations).item() * 100, decimals=4)

# Display results
print("Prior Mean:")
for i, ticker in enumerate(['GOOGL', 'AAPL', 'ADBE', 'META', 'MSFT']):
    print(f"{ticker}: {means_prior[0, i] * 100:.2f}")

print("\nPrior Volatilities:")
for i, ticker in enumerate(['GOOGL', 'AAPL', 'ADBE', 'META', 'MSFT']):
    print(f"{ticker}: {vols_prior[0, i] * 100:.2f}")

print("\nPosterior Mean:")
for i, ticker in enumerate(['GOOGL', 'AAPL', 'ADBE', 'META', 'MSFT']):
    print(f"{ticker}: {posterior_mean[0, i]:.2f}")

print("\nPosterior Volatilities:")
for i, ticker in enumerate(['GOOGL', 'AAPL', 'ADBE', 'META', 'MSFT']):
    print(f"{ticker}: {posterior_vols[i]:.2f}")

/usr/local/lib/python3.10/dist-packages/entropy_pooling/entropy_pooling.py:67: RuntimeWarning: overflow encountered in exp
  x = np.exp(log_x)
/usr/local/lib/python3.10/dist-packages/entropy_pooling/entropy_pooling.py:68: RuntimeWarning: invalid value encountered in matmul
  gradient = rhs - lhs @ x
/usr/local/lib/python3.10/dist-packages/entropy_pooling/entropy_pooling.py:69: RuntimeWarning: invalid value encountered in matmul
  objective = x.T @ (log_x - log_p) - lagrange_multipliers.T @ gradient


Prior Mean:
GOOGL: 0.00
AAPL: 0.00
ADBE: 0.00
META: 0.00
MSFT: 0.00

Prior Volatilities:
GOOGL: 0.02
AAPL: 0.02
ADBE: 0.02
META: 0.02
MSFT: 0.01

Posterior Mean:
GOOGL: 0.00
AAPL: 0.00
ADBE: 0.00
META: 0.00
MSFT: 0.00

Posterior Volatilities:
GOOGL: 0.00
AAPL: 0.00
ADBE: 0.00
META: 0.00
MSFT: 0.00


In [ ]:

# Q matrix (posterior probabilities)
print("\nQ Matrix (Posterior Probabilities):")
print(posterior_prob)
print("prior mean",means_prior)
posterior_prob.shape


Q Matrix (Posterior Probabilities):
[[0.]
 [0.]
 [0.]
 ...
 [0.]
 [0.]
 [0.]]
prior mean [[3.06815841e-05 3.51718338e-05 9.34218603e-06 1.05790647e-06
  1.12277120e-05]]


(5000, 1)

In [ ]:
print("A's shape",A.shape)
print("b's shape",b.shape)
print("G's shape",G.shape)
print("h's shape",h.shape)
print("R's shape",R.shape)

A's shape (3, 5000)
b's shape (3, 1)
G's shape (2, 5000)
h's shape (2, 1)
R's shape (5000, 5)
